# Intro ás Listas

- **LazyColumn** - Lista vertical
    - Monta apenas componentes que esta visivél para usuário
    - Desenha conforme o usuário precisa

- **LazyRow** - Lista horizontal
    - Monta apenas componentes que esta visivél para usuário
    - Desenha conforme o usuário precisa

## Items das Lazy's

Ao criar estes componentes Lazy's, temos a opção de o parametros **items** e junto a ele a contagem de itens a inserir.

- Além de passagem o numero, valor que representa a quantidade de repetição do item, podemos também passar um lista

- Além do numero de iterações que pode ser definido ou carregado uma lista, atribuimos uma função a cada laço, podendo receber, text, card entre outros

In [ ]:
LazyRow(){
    items(gamesListState){
        StudioCard(game = it)
    }
}
Spacer(modifier = Modifier.height(16.dp))
LazyColumn(){
    items(gamesListState){
        GameCard(game = it)
    }

## Filtro

Iremos inserir um campo de busca, onde apos inserir os dados e pesquisar, ele faça uma filtragem dos dados.

Para isso, usamos a função listaQualquer.filter, filtro este que pode ser, inúmero:

- StartsWith, começar com algum prefixo, ignoreCase ou não.
- EndsWith, finaliza com algum sufixo, ignoreCase ou não.

- Adiciona a função ao click do button

In [ ]:
//Função getallGamesByStudio do tipo studio String, que retorna um lista de games
fun getGamesByStudio(studio: String): List<Game>{
    return getAllGames().filter {
//        Retorna uma lista de games cujo nome do estúdio deve começar com studio, ignorando os cases
        it.studio.startsWith(prefix = studio, ignoreCase = true)
    }
}

# Intro ao consumo de APIS e Retrofit

- App conectada, para ser conectada, precisamos se conectar com algum serviço, ou seja um serviço back-end que esteja rodando a um servidor remoto, que atende requisições http

- API Rest, são as APIS mais comuns, onde temos um url no formato http que faz a requisição de uma chamada, onde geralmente os serviços nos devolvem em formato JSON ou XML. Onde estes trabalham com chave:valor 

- Mas mesmo tendo este arquivo json com chave:valor, precisamos converter de objeto para texto e como fazemos de texto para objeto.

- Para isso temos a biblioteca Retrofit, que é uma padrão de consumo de API, onde o retrofit abstrai uma serie de funcionalidade para nós

## Configuração do ambiente para retrofit

- Retrofit não faz parte do pacote, por tanto precisamos importa-la e configura-la.


- No Gradle Scripts > build.gradle(Module:app)> na sessão de dependencias insira:

    <code>implementation 'com.squareup.retrofit2:retrofit:2.9.0'</code>

    <code>implementation 'com.squareup.retrofit2:rconverter-gson:2.9.0'</code>

- APIS retornam normalmente arquivo texto no formato JSON, quem converte e esse JSON em objeto

- Criamos um pacote chamado "service", para guardar a classe que faz a conexão com o servidor remoto

In [ ]:
// Ou podemos usar a implementação da seguinte maneira:

//Dependencias do retrofit
implementation ("com.squareup.retrofit2:retrofit:2.9.0")
implementation ("com.squareup.retrofit2:converter-gson:2.9.0")
//implementation ("com.google.code.gson:gson:2.9.0")

# Padrão Json

- Normalmente o retorno dos API'S são em arquivos JSON, por exemplo o **APIS VIACEP**, onde usamos através do http https://viacep.com.br/ws/xxxxxx/json/, fazemos uma requisição entre clienteWeb - Servidor. e o servidor nos retorna um arquivo JSON e não uma pagina comum.

- Onde substituimos o "xxxxxx" por um **CEP** e nos é retornado um arquivo do tipo JSON com dados do CEP.


Além do padrão de requisição HTTP acima: 

 <code>https://viacep.com.br/ws/CEPQUALQUER/json/</code> 

 <code>https://viacep.com.br/ws/09051610/json/</code> 


 Podemos usar o padrão que pesquisa por Estado/Cidade/rua, da seguinte maneira:

 <code>https://viacep.com.br/ws/ESTADO/CIDADE/RUA/json/</code> 
 
 <code>https://viacep.com.br/ws/SP/SantoAndre/Inverno/json/</code>


Exemplo do resultado:

In [ ]:
{
    "cep": "09051-610",
    "logradouro": "Rua Inverno",
    "complemento": "",
    "bairro": "Jardim Bom Pastor",
    "localidade": "Santo André",
    "uf": "SP",
    "ibge": "3547809",
    "gia": "6269",
    "ddd": "11",
    "siafi": "7057"
}

- Para que o Android possa receber estes dados, precisamos criar um **classe Kotlin** que saiba receber estes dados e representa-la.

- Lembrando que podemos criar uma classe que pegue apenas algum dos dados do Json e não todos que a API fornece!

- Os atributos da classe precisam ter o nome igual da chave do JSON, caso contrario precisaremos usa o **@SerializedName**

- Na construção da classe, se quisermos mudar o nome do atributo, por exemplo de "logradouro" para "rua" podemos usar o **@SerializedName("logradouro")**


In [ ]:
package br.com.fiap.consultacep.model

import com.google.gson.annotations.SerializedName


data class Endereco(
    val cep: String = "",
    @SerializedName("logradouro") val rua: String = "",
    @SerializedName("localidade") val cidade: String = "",
    val bairro: String = "",
    val uf: String = "",

)

## Criação da interface de requisição á API

- Criamos a interface dentro do pacote "Service", e o arquivo será do tipo interface

- Criamos neste arquivo uma função do tipo Call< Endereco >

- A cima desta função adicionamos o decorator @GET (do import retrofit) e parametro do get colocamos o endpoint com a variavel

- @Path, indica que a chave que se encontra na URL deverá ser substituidá pelo valro do argumanento passado na chamado do método

Exemplo da interface service do tipo interface:

In [ ]:
interface CepService {


    //    Inicio da construção dos dois métodos get
    
    //    Pega endereço usando cep
        //https://viacep.com.br/ws/01001000/json/
        @GET("{cep}/json/")
        fun getEndereco(@Path("cep") cep: String): Call<Endereco>
    
    //    Pega o cep, usando uf, cidade,rua
        //https://viacep.com.br/ws/RS/Porto%20Alegre/Domingos/json/
        @GET("{uf}/{cidade}/{rua}/json/")
        fun getEnderecos(
            @Path("uf") uf: String,
            @Path("cidade") cidade: String,
            @Path("rua") rua: String
        ): List<Call<Endereco>>
    
    }

## Criação do cliente HTTP do retrofit

- Retrofit fará o papel de cliente, ele que sabe fazer as requisições HTTP

- Agora iremos configurar este serviço, para isso precisamos criar, dentro do pacote 'service" um arquivo tipo class com sufixo factory

- Nesta classe precisamos guardar em uma variavel a BASE HTTP

- Criaremos tambem um variavel retrofitFactory que recebe um Retrofit, com :
    - .Builder()
    - .BaseURl(URL)
    - .AddConverterFactory(GsonConverterFacotory.create())
    - .build()

- Criamos tambem um implementação da função, que era interface no arquivo xxxService

In [ ]:
class RetrofitFactory {
    //    Armazena a URL BASE
        private val URL = "https://viacep.com.br/ws/"
    
        private val retrofitFactory = Retrofit
            .Builder()
            .baseUrl(URL)
            .addConverterFactory(GsonConverterFactory.create())
            .build()
    
        fun getCepService(): CepService {
            return retrofitFactory.create(CepService::class.java)
        }
    }

## Executando chamadas para a API

CallBack do retrofit2

**OnFailure** é quando não conseguimos se comunicar com o servidor

**OnResponse**, quando tivemos a resposta de algum problema

//Retrofit, havia dado problema, pois em ServiceCep.kt, eu havia colocado o tipo de retorno da função getEnderecos como:

 <code>< List < Call< Endereco >>> </code>

 E deveria ser

 <code>< Call < List< Endereco >>> </code>



In [ ]:
fun getEnderecos(
    @Path("uf") uf: String,
    @Path("cidade") cidade: String,
    @Path("rua") rua: String
): Call<List<Endereco>>

## Permitindo conexão com a internet

Por padrão o android proibe os apps de conectarem-se a internet, para solucionar isso precisamos fazer algumas alterações:

- Vamos na pasta> app> manifests> e adicionaremos antes da sessão "Application" o código:

    <code> 
    < uses-permission android:name="android.permission INTERNET" />

    </code>

